# T7 - Calibration

We saw in Tutorial 4 how to load and plot data. But the next step is to actually *calibrate* the model to the data, i.e. find the model parameters that are the most likely explanation for the observed data. This tutorial gives an introduction to the Fit object and some recipes for optimization approaches.

## The Fit object

The Fit object is responsible for quantifying how well a given model run matches the data. Let's consider a simple example, building on Tutorial 4:

In [ ]:
import covasim as cv
cv.options.set(dpi=100, show=False, close=True, verbose=0) # Standard options for Jupyter notebook

pars = dict(
    pop_size = 10_000,
    start_day = '2020-02-01',
    end_day   = '2020-04-11',
    beta      = 0.015,
)
sim = cv.Sim(pars=pars, datafile='example_data.csv', interventions=cv.test_num(daily_tests='data'))
sim.run()
sim.plot(to_plot=['cum_tests', 'cum_diagnoses', 'cum_deaths'])

We can see that tests match extremely well (they're input data!), diagnoses match reasonably well, and deaths match poorly. Can the Fit object capture our intuition about this?

In [ ]:
fit = sim.compute_fit()
print(fit.mismatches)
print(fit.mismatch)

So the results seem to match our intuition. (Note that by default the Fit object uses normalized absolute difference, but other estimates, such as mean squared error, are also possible.)

What if we improve the fit? Does the mismatch reduce?

In [ ]:
sim['rel_death_prob'] = 2 # Double the death rate since deaths were too low
sim.initialize(reset=True) # Reinitialize the sim

# Rerun and compute fit
sim.run()
fit = sim.compute_fit()

# Output
sim.plot()
fit.plot()
print(fit.mismatches)
print(fit.mismatch)

As expected, the fit is improved.

## Calibration approaches

Calibration is a complex and dark art and cannot be covered fully here; many books have been written about it and it continues to be an area of active research. A good review article about calibrating agent-based models like Covasim is available [here](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1007893). Calibration is usually expressed as an optimization problem: specifically, find a vector of parameters *θ* that minimizes the mismatch between the data *D* and the model *M(θ)*.

In practice, most calibration is done simply by hand, as in the example above. Once deaths are "calibrated", the user might modify testing assumptions so that the diagnoses match. Since we are only fitting to deaths and diagnoses, the model is then "calibrated".

However, automated approaches to calibration are possible as well. The simplest is probably the built-in SciPy optimization functions, e.g. `scipy.optimize`. A wrinkle here is that normal gradient descent methods **will not work** with Covasim or other agent-based models, due to the stochastic variability between model runs that makes the landscape very "bumpy". One way of getting around this is to use many different runs and take the average, e.g.:

```python
import covasim as cv
import numpy as np
import scipy

def objective(x, n_runs=10):
    print(f'Running sim for beta={x[0]}, rel_death_prob={x[1]}')
    pars = dict(
        pop_size       = 10_000,
        start_day      = '2020-02-01',
        end_day        = '2020-04-11',
        beta           = x[0],
        rel_death_prob = x[1],
        verbose        = 0,
    )
    sim = cv.Sim(pars=pars, datafile='/home/cliffk/idm/covasim/docs/tutorials/example_data.csv', interventions=cv.test_num(daily_tests='data'))
    msim = cv.MultiSim(sim)
    msim.run(n_runs=n_runs)
    mismatches = []
    for sim in msim.sims:
        fit = sim.compute_fit()
        mismatches.append(fit.mismatch)
    mismatch = np.mean(mismatches)
    return mismatch

guess = [0.015, 1] # Initial guess of parameters -- beta and relative death probability
pars = scipy.optimize.minimize(objective, x0=guess, method='nelder-mead') # Run the optimization
```

This should converge after roughly 3-10 minutes, although you will likely find that the improvement is minimal.

What's happening here? Trying to overcome the limitations of an algorithm that expects deterministic results simply by running more sims is fairly futile – if you run *N* sims and average them together, you've only reduced noise by √*N*, i.e. you have to average together 100 sims to reduce noise by a factor of 10, and even that might not be enough. Clearly, we need a more powerful approach.

## Built-in calibration

One such package we have found works reasonably well is called [Optuna](https://optuna.org/). It is built into Covasim as `sim.calibrate()` (it's not installed by default, so please install it first with `pip install optuna`). Do not expect this to be a magic bullet solution: you will likely still need to try out multiple different parameter sets for calibration, manually update the values of uncalibrated parameters, check if the data actually make sense, etc. Even once all these things are in place, it still needs to be run for enough iterations, which might be a few hundred iterations for 3-4 calibrated (free) parameters or tens of thousands of iterations for 10 or more free parameters. The example below should get you started, but best to expect that it will _not_ work for your particular use case without significant modification!

In [ ]:
'''
Example for running built-in calibration with Optuna
'''

import sciris as sc
import covasim as cv

# Create default simulation
pars = sc.objdict(
    pop_size       = 10_000,
    start_day      = '2020-02-01',
    end_day        = '2020-04-11',
    beta           = 0.010,
    rel_death_prob = 1.0,
    interventions  = cv.test_num(daily_tests='data'),
    verbose        = 0,
)
sim = cv.Sim(pars=pars, datafile='example_data.csv')

# Parameters to calibrate -- format is best, low, high
calib_pars = dict(
    beta           = [pars.beta, 0.005, 0.20],
    rel_death_prob = [pars.rel_death_prob, 0.5, 3.0],
)

if __name__ == '__main__':

    # Run the calibration
    n_trials = 25
    n_workers = 4
    calib = sim.calibrate(calib_pars=calib_pars, n_trials=n_trials, n_workers=n_workers)

So it improved the fit (see above), but let's visualize this as a plot:

In [ ]:
# Plot the results
calib.plot(to_plot=['cum_tests', 'cum_diagnoses', 'cum_deaths'])

Compared to `scipy.optimize.minimize()`, Optuna took less time and produced a much better fit. However, it's still far from perfect -- more iterations, and calibrating more parameters beyond just these two, would still be required before the model could be considered "calibrated".